## Выполнили: Жерноклеев Дмитрий Дамирович, Индык Илья Андреевич, Реентович Александр Анатольевич

# Домашнее задание
## Harry Potter and the Action Prediction Challenge from Natural Language

*deadline*: 14 ноября 2022, 23:59

В этом домашнем задании вы будете работать с корпусом Harry Potter and the Action Prediction Challenge. Корпус собран из фанфиков о Гарри Поттере и состоит из двух частей: 1) сырые тексты, 2) фрагменты текстов, описывающих ситуацию, в которой произнесено заклинание.

Корпус описан в статье: https://arxiv.org/pdf/1905.11037.pdf

David Vilares and Carlos Gómez-Rodríguez. Harry Potter and the Action Prediction Challenge from Natural Language. 2019 Annual Conference of the North American Chapter of the Association for Computational Linguistics. To appear.

Код для сбора корпуса находится в репозитории: https://github.com/aghie/hpac . Корпус можно скачать по инструкции из этого репозитория, но для экономии времени авторы задания уже скачали и подготовили данные к работе. 

Ссылки на собранный корпус: 
* Сырые тексты:  https://www.dropbox.com/s/23xet9kvbqna1qs/hpac_raw.zip?dl=0
* Токенизированные тексты в нижнем регистре: https://www.dropbox.com/s/gwfgmomdbetvdye/hpac_lower_tokenized.zip?dl=0
* train-test-dev: https://www.dropbox.com/s/3vdz0mouvex8abd/hpac_splits.zip?dl=0

Части 1, 2 задания должны быть выполнены на полных текстах (сырых или предобработанных -- на ваше усмотрение), Часть 3 – на разбиение на тестовое, отладочное и обучающее множества. Тестовое множество должно быть использовано исключительно для тестирования моделей, обучающее и отладочное – для выбора модели и параметров. 

В статье и репозитории вы найдете идеи, которые помогут вам выполнить домашнее задание. Их стоит воспринимать как руководство к действию, и не стоит их копировать и переиспользовать. Обученные модели использовать не нужно, код для их обучения можно использовать как подсказку. 

## ПРАВИЛА
1. Домашнее задание выполняется в группе до 3-х человек.
2. Домашнее задание оформляется в виде отчета либо в .pdf файле, либо ipython-тетрадке. 
3. Отчет должен содержать: нумерацию заданий и пунктов, которые вы выполнили, код решения, и понятное пошаговое описание того, что вы сделали. Отчет должен быть написан в академическом стиле, без излишнего использования сленга и с соблюдением норм русского языка.
4. Не стоит копировать фрагменты лекций, статей и Википедии в ваш отчет.
5. Отчеты, состоящие исключительно из кода, не будут проверены и будут автоматически оценены нулевой оценкой.
6. Плагиат и любое недобросоветсное цитирование приводит к обнуление оценки. 

In [1]:
!pip install gensim --upgrade -q  # апгрейд пакета генсим

     |████████████████████████████████| 24.1 MB 1.4 MB/s 


In [2]:
import pandas as pd  # импортируем необходимые для выполнения задания модули
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re
from tqdm import tqdm
import urllib.request
import gensim
from gensim.models import word2vec
from gensim.test.utils import datapath
from gensim import utils
from google.colab import files
from sklearn.manifold import TSNE
import plotly.express as px

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!wget https://www.dropbox.com/s/12toaj67fjrguhd/hpac_raw.zip -q
!wget https://www.dropbox.com/s/1ndto6dce5wg7j2/hpac_lower_tokenized.zip -q
!wget https://www.dropbox.com/s/ftinwwjfyydevth/hpac_splits.zip -q

### Данные
Сырые тексты 

Подгрузим наши тексты

In [ ]:
!unzip -q hpac_lower_tokenized
!unzip -q hpac_splits -d /content/hpac_splits
!unzip -q hpac_raw -d /content/hpac_raw

In [ ]:
!ls hpac_source | wc -l

36225


In [ ]:
!ls hpac_splits | wc -l

2


In [ ]:
!ls hpac_raw | wc -l

1


train, test, dev файлы

In [ ]:
df_train = pd.read_csv('hpac_splits/hpac_corpus/hpac_training_128.tsv', sep='\t', header=None)
df_test = pd.read_csv('hpac_splits/hpac_corpus/hpac_test_128.tsv', sep='\t', header=None)
df_dev = pd.read_csv('hpac_splits/hpac_corpus/hpac_dev_128.tsv', sep='\t', header=None)

In [ ]:
df_train.head()

,0,1,2
0,7642954.0.676,RIDDIKULUS,were staring at her . she was up next to face ...
1,10443333.0.5753,RIDDIKULUS,"that whole time . her first reaction , for whi..."
2,4703706.0.8690,STUPEFY,we watched his inglorious withdrawal together ...
3,4593427.0.1815,ACCIO,"my wand , `` incendio . '' this wretched chill..."
4,4278446.0.2692,EXPELLIARMUS,"already compared ours , they 're the same ever..."


In [ ]:
print(df_train.iloc[0][1], '\n', df_train.iloc[0][2])

RIDDIKULUS 
 were staring at her . she was up next to face the boggart in defense against the dark arts class . she was not scared , but what she was worried about was what had happened with lysander . she looked up at the boggart in front of her which had previously been a humongous spider . its eyes locked on her . before she could think of what frightened her , the spider transformed into lysander . he was dying . there were giggles coming from the male and female hufflepuff students . there was a smirk on lorcan 's face . `` lily help me '' i ca n't fail this class because of a secret love . lily lifted her wand and said , ``


### Как использовать WordNet из nltk?

In [ ]:
# скачиваем WordNet
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# слово -> множество синсетов (синонимов разных смыслов исходного слова)
wn.synsets('magic')

[Synset('magic.n.01'), Synset('magic_trick.n.01'), Synset('charming.s.02')]

In [ ]:
# посмотрим, что внутри одного синсета
wn.synsets('magic')[1].lemmas()[0]

Lemma('magic_trick.n.01.magic_trick')

In [ ]:
# возьмем лемму одного из слов из синсета
wn.synsets('magic')[1].lemmas()[-1].name()

'deception'

## Часть 1. [2 балла] Эксплоративный анализ 
1. Найдите топ-1000 слов по частоте без учета стоп-слов.
2. Найдите топ-10 по частоте: имен, пар имя + фамилия, пар вида ''профессор'' + имя / фамилия. 

[бонус] Постройте тематическую модель по корпусу HPAC.

[бонус] Найдите еще что-то интересное в корпусе (что-то специфичное для фанфиков или фентези-тематики)

### 1.1

In [ ]:
with open('/content/hpac_source/10000036', 'r') as f:
  print(len(f.readlines()))
  f.seek(0)
  for l in f:
    print(l)

1
first , harry heard a faint cursing , but this was nothing unusual and so he rubbed his hair dry , without ruffles or excitement . to the following sigh he also gave no attention , impassive he started , to get dressed . but after a particularly deep sigh , he just had the first buttons of his shirt buttoned , his curiosity was piqued . `` okay , what 's going on ? '' ron , from whom he could see only the bare lower legs and feet behind the locker door , growled : `` nothing ! '' `` your behavior speaks another language , mate . you are usually always the first , who disappear from the locker room - '' he poked his head around the door . `` now you 're still only in a towel . and why the hell are you staring in your locker ? '' he peered over ron 's shoulder into the locker . `` it is not about a spider -- or ? '' ron blocked with his full height a sight into the locker and snorted : `` you think , then i would still be here ?! '' harry turned with a grin back to his own locker . acc

In [ ]:
nltk.download('stopwords')  # скачаем стоп-слова для английского языка
print(stopwords.words('english'))
stop = stopwords.words('english') + list(punctuation) + ['``', '/n', '--', '...']

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Считаем корпус наших текстов в удобной форме для подачи в функцию, которая выполнит предобработку (препроцессинг) текстов и подсчет встречаемости (частотности) слов.

In [ ]:
d = {}  # создадим словарь из пар название файла - текст, лежащий в этом файле
dir_name = '/content/hpac_source'
for f_name in os.listdir(dir_name):
  with open(dir_name + '/' + f_name, 'r') as f:
    l = f.readline()
    d[f_name] = l

In [ ]:
# Создадим датафрейм из нашего словарика
df_txt = pd.DataFrame(d.items(), columns=['f_name', 'text'])
df_txt

,f_name,text
0,8451124,the kiss hermione sighs . it 's been days sinc...
1,9101843,this was just a pointless essay i wrote awhile...
2,7791855,author 's note : just a little something for f...
3,2173706,ok this is evil monkeys . i wrote at school an...
4,12583165,this little drabble was written for the prompt...
...,...,...
36220,5471287,title : afterwards character -LRB- s -RRB- : d...
36221,6039351,things i am not allowed to do at hogwarts i wi...
36222,11726641,when what people were optimistically describin...
36223,3850342,disclaimer : definitely not mine . belongs to ...


In [ ]:
def preprocess_text(text):  
    """функция предобработки текста"""
    text = re.sub(r'/n', ' ', text.lower())
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

In [ ]:
# Воспользуемся функцией CountVectorizer для подсчета частоты слов
vec = CountVectorizer(stop_words=stop, preprocessor=preprocess_text)
bow = vec.fit_transform(df_txt.text)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arent', 'couldnt', 'didnt', 'doesnt', 'dont', 'hadnt', 'hasnt', 'havent', 'isnt', 'mightnt', 'mustnt', 'neednt', 'shant', 'shes', 'shouldnt', 'shouldve', 'thatll', 'wasnt', 'werent', 'wont', 'wouldnt', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  % sorted(inconsistent)


In [ ]:
bow.shape

(36225, 552117)

In [ ]:
len(vec.vocabulary_)

552117

In [ ]:
df_freq = pd.DataFrame({'word': vec.get_feature_names_out(), 'freq': np.array(bow.sum(axis=0)).squeeze(0)})
df_freq = df_freq.sort_values(by='freq', ascending=False).iloc[:1000].reset_index(drop=True)
df_freq

,word,freq
0,nt,4396854
1,harry,3991215
2,said,2262083
3,would,1903195
4,hermione,1827016
...,...,...
995,umbridge,41609
996,unless,41508
997,shop,41435
998,tight,41321


In [4]:
df_freq.head(20)

,word,freq
0,nt,4396854
1,harry,3991215
2,said,2262083
3,would,1903195
4,hermione,1827016
5,could,1687870
6,back,1396466
7,draco,1386279
8,one,1376341
9,like,1256577


In [ ]:
df_freq.to_csv('hpac_top1k_fix.csv')  # сохраним результаты п. 1.1 задания
files.download('/content/hpac_top1k_fix.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


## Часть 2. [2 балла] Модели представления слов 
Обучите модель представления слов (word2vec, GloVe, fastText или любую другую) на материале корпуса HPAC.
1. Продемонстрируйте, как работает поиск синонимов, ассоциаций, лишних слов в обученной модели. 
2. Визуализируйте топ-1000 слов по частоте без учета стоп-слов (п. 1.1) с помощью TSNE или UMAP (https://umap-learn.readthedocs.io).

### 2.1

In [ ]:
def clear_text(txt, f_id, dir, stop_w=stop, rm_stop=True):
    """Функция создания файла, содержащего весь корпус в обработанном виде."""
    txt = re.sub('/n', '', txt)  # предобработка, фактически, не изменилась.
    txt = re.sub('[^a-zA-Z\s]', '', txt)
    # приводим к нижнему регистру и разбиваем на слова по символу пробела
    txt = txt.lower().split()
    if rm_stop: # убираем стоп-слова
        words = [w for w in txt if not w in stop_w]
    with open(os.path.join(dir, 'corp.txt'), 'a') as f:  
      f.write(' '.join(words) + '\n')  # пишем очередную строку в файл

In [ ]:
class MyCorpus:
    """Класс итератора, который выдает предложения как список строк."""

    def __init__(self, path):
      self.path = path  # путь до нашего файла

    def __iter__(self):
        corpus_path = datapath(self.path)
        for line in open(corpus_path):
            yield line.split()  # считываем строчку и деим по пробелам

Теперь после всей предварительной работы можем обучить нашу модель word2vec из gensim.

In [ ]:
sentences = MyCorpus(os.path.join(os.getcwd(), 'corp.txt'))
model = word2vec.Word2Vec(sentences=sentences, workers=4, 
                          vector_size=250, min_count=12, window=4)

In [ ]:
len(model.wv)  # количество слов в словаре модели (мы выкинули все, которые встречаются < 12 раз)

84311

In [ ]:
w2v_freq_df = pd.DataFrame(model.wv.key_to_index.keys(), 
                           columns=['word'])[:1000]
w2v_freq_df  # это вспомогательный датафрейм для проверки корректности решения п. 1.1

,word
0,nt
1,harry
2,said
3,would
4,hermione
...,...
995,umbridge
996,unless
997,shop
998,tight


In [ ]:
(w2v_freq_df.word == df_freq.word).sum()

998

In [ ]:
'muggle' in model.wv.key_to_index 

True

Покажем, как работает поиск синонимов, ассоциаций и лишних слов в нашей модели.

In [ ]:
print('Synonyms:')
print(model.wv.most_similar('harry', topn=3))
print(model.wv.most_similar('wizard', topn=3))
print(model.wv.most_similar('spell', topn=3))

print('Analogies:')
print(model.wv.most_similar(positive=['harry', 'girl'], negative=['boy'], topn=2))
print(model.wv.most_similar(positive=['dumbledore', 'woman'], negative=['man'], topn=2))
print(model.wv.most_similar(positive=['wizard', 'woman'], negative=['man'], topn=2))
print(model.wv.most_similar(positive=['school', 'magic'], negative=['ordinary'], topn=2))

print('Doesn\'t match:')
print('magic spell wizard ron => Extra word:', 
      model.wv.doesnt_match('magic spell wizard ron'.split()))
print('voldemort draco snape stick => Extra word:', 
      model.wv.doesnt_match('voldemort draco snape stick'.split()))
print('voldemort draco snape vernon => Extra word:', 
      model.wv.doesnt_match('voldemort draco snape vernon'.split()))

Synonyms:
[('hermione', 0.7431302070617676), ('draco', 0.6963883638381958), ('ginny', 0.6807665824890137)]
[('witch', 0.692665696144104), ('man', 0.648740828037262), ('wizards', 0.6444472670555115)]
[('spells', 0.7912835478782654), ('charm', 0.7791950702667236), ('counterspell', 0.7685550451278687)]
Analogies:
[('hermione', 0.8361391425132751), ('ginny', 0.809330403804779)]
[('mcgonagall', 0.717517614364624), ('minerva', 0.6930954456329346)]
[('witch', 0.809012770652771), ('wizards', 0.5605471134185791)]
[('hogwarts', 0.5735276341438293), ('magics', 0.4711984395980835)]
Doesn't match:
magic spell wizard ron => Extra word: ron
voldemort draco snape stick => Extra word: stick
voldemort draco snape vernon => Extra word: vernon


In [ ]:
print('One more great analogy:')
print(model.wv.most_similar(positive=['azkaban', 'school'], negative=['prison'], topn=1))

Analogies:
[('hogwarts', 0.7566561698913574)]


По результатам выше можно сказать, что все результаты адекватны. Теперь сохраним нашу модель.

In [ ]:
path_mod = os.path.join(os.getcwd(), 'hpac_w2v.model')
print("Saving model...")
model.save(path_mod)
print(f'Successfully saved to {path_mod}!')

Saving model...
Successfully saved to /content/hpac_w2v.model!


In [ ]:
files.download(os.path.join(os.getcwd(), 'hpac_w2v.model')) 
files.download('/content/hpac_w2v.model.wv.vectors.npy')
files.download('/content/hpac_w2v.model.syn1neg.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 2.2

Снова скачаем нашу модель и перейдем к визуализации.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q /content/drive/MyDrive/Colab\ Notebooks/NLP/w2v_model.zip -d /content/

In [ ]:
model = word2vec.Word2Vec.load('/content/hpac_w2v.model')

In [ ]:
df_freq.sample(5)

,word,freq
830,soul,50714
450,speak,96696
42,two,562470
293,onto,143488
120,put,280582


In [ ]:
  # отберем из словаря топ-слова по результатам п. 1.1 и получим из модели их эмбеддинги
top_words = [w for w in df_freq.word if w in model.wv]
top_words_vec = np.array([model.wv[w] for w in top_words])

In [ ]:
# исользуем tsne для получения низкоразмерных представлений
tsne = TSNE(n_components=2, learning_rate='auto',
            init='random', random_state=42)
top_words_tsne = tsne.fit_transform(top_words_vec)

In [ ]:
df_tsne = pd.DataFrame(top_words_tsne, columns=['x', 'y'])
df_tsne['word'] = top_words
df_tsne

,x,y,word
0,3.901484,8.343550,nt
1,-20.104765,15.605262,harry
2,-29.642834,5.513092,said
3,7.964181,16.549751,would
4,-25.847094,19.560932,hermione
...,...,...,...
995,-20.010830,21.686295,umbridge
996,8.387957,6.505520,unless
997,35.565605,-22.470924,shop
998,-1.535295,-35.068943,tight


In [ ]:
# непосредственно интерактивная визуализация топ-1000 векторов на плоскости
fig = px.scatter(df_tsne, x='x', 
                y='y', text='word')

fig.update_traces(textposition='top center')

fig.update_layout(
    autosize=False,
    height=600,
    width=900,
    title_text='t-SNE for w2v model on HPAC'
)
fig.show()

Из данной визуализации видно, что похожие слова (конечно, подразумеваются те, которые употребляются в одинаковом контексте) расположены близко друг к другу, как и ожидается на выходе w2v.

## Часть 3. [5 баллов] Классификация текстов
Задача классификации формулируется так: данный фрагмент фанфика описывают какую-то ситуацию, которая предшествует произнесению заклинания. Требуется по тексту предсказать, какое именно заклинание будет произнесено. Таким образом, заклинание - это фактически метка класса. Основная мера качества – macro $F_1$.
Обучите несколько классификаторов и сравните их между собой. Оцените качество классификаторов на частых и редких классах. Какие классы чаще всего оказываются перепутаны? Связаны ли ошибки со смыслом заклинаний?

Используйте фрагменты из множества train для обучения, из множества dev для отладки, из множества test – для тестирования и получения итоговых результатов. 

1. [1 балл] Используйте fastText в качестве baseline-классификатора.
2. [2 балла] Используйте сверточные  или реккурентные сети в качестве более продвинутого классификатора. Поэкспериментируйте с количеством и размерностью фильтров, используйте разные размеры окон, попробуйте использовать $k$-max pooling. 
3. [2 балла] Попробуйте расширить обучающее множество за счет аугментации данных. Если вам понадобится словарь синонимов, можно использовать WordNet (ниже вы найдете примеры).

[бонус] Используйте результат max pooling'а как эмбеддинг входного текста. Визуализируйте эмбеддинги 500-1000 предложений из обучающего множества и изучите свойства получившегося пространства.

[бонус] Используйте ваш любимый классификатор и любые (честные) способы повышения качества классификации и получите macro $F_1$ больше 0.5.

## Часть 4. [1 балл] Итоги
Напишите краткое резюме проделанной работы. Читали ли вы сами Гарри Поттера или фанфики о нем и помогло ли вам знание предметной области в выполнении домашнего задания?